In [1]:
!python3 -m spacy download el
!python3 -m spacy download es

⚠ As of spaCy v3.0, shortcuts like 'el' are deprecated. Please use the
full pipeline package name 'el_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 31.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('el_core_news_sm')
⚠ As of spaCy v3.0, shortcuts like 'es' are deprecated. Please use the
full pipeline package name 'es_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 33.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [2]:
!pip install torchtext
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 17.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.8 MB/s eta 0:00:0000:01


In [3]:
!pip install torch==2.3.0+cpu torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

Looking in indexes: https://download.pytorch.org/whl/cpu
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.4/190.4 MB 6.1 MB/s eta 0:00:0000:0100:01
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 1.5 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.7 MB/s eta 0:00:00a 0:00:01m
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: 

In [4]:
import os
import random
from tqdm import tqdm
from dataclasses import dataclass

import pandas as pd
import numpy as np

import spacy
import es_core_news_sm
import el_core_news_sm

import datasets
import torchtext

import torch
import torch.nn as nn
import torch.optim as optim

import evaluate

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import warnings
warnings.filterwarnings("ignore")

In [5]:
seed = 42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [6]:
dataset = datasets.load_dataset("Helsinki-NLP/europarl", 'el-es')

dataset

README.md:   0%|          | 0.00/76.3k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/155M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/156M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1272383 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1272383
    })
})

In [7]:
num_rows = dataset['train'].num_rows

# third_size = num_rows // 3

# dataset['train'] = dataset['train'].select(range(2 * third_size, num_rows))
size = 20000

dataset['train'] = dataset['train'].select(range(0, size))

dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 20000
    })
})

In [8]:
train_valid_test = dataset['train'].train_test_split(test_size=0.2)

train_valid = train_valid_test['train'].train_test_split(test_size=0.2)

train_dataset = train_valid['train']
valid_dataset = train_valid['test']
test_dataset = train_valid_test['test']

print(f"Train set size: {train_dataset.num_rows}")
print(f"Validation set size: {valid_dataset.num_rows}")
print(f"Test set size: {test_dataset.num_rows}")
train_dataset

Train set size: 12800
Validation set size: 3200
Test set size: 4000


Dataset({
    features: ['translation'],
    num_rows: 12800
})

In [9]:
def restructure_features(example):
    return {
        "el": example["translation"]['el'],
        "es": example["translation"]['es']
    }

In [10]:
train_data = train_dataset.map(restructure_features, remove_columns=["translation"])
valid_data = valid_dataset.map(restructure_features, remove_columns=["translation"])
test_data = test_dataset.map(restructure_features, remove_columns=["translation"])

Map:   0%|          | 0/12800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [11]:
def tokenize_example(
    example,
    el_nlp,
    es_nlp,
    max_length,
    lower,
    sos_token,
    eos_token
):
    el_tokens = [token.text for token in el_nlp.tokenizer(example['el'])][:max_length]
    es_tokens = [token.text for token in es_nlp.tokenizer(example['es'])][:max_length]
    if lower:
        el_tokens = [token.lower() for token in el_tokens]
        es_tokens = [token.lower() for token in es_tokens]
    el_tokens = [sos_token] + el_tokens + [eos_token]
    es_tokens = [sos_token] + es_tokens + [eos_token]
    return {"el_tokens": el_tokens, "es_tokens": es_tokens}

In [12]:
el_nlp = el_core_news_sm.load()
es_nlp = es_core_news_sm.load()

In [13]:
max_length = 1_000
lower = True
sos_token = "<sos>"
eos_token = "<eos>"

fn_kwargs = {
    "el_nlp": el_nlp,
    "es_nlp": es_nlp,
    "max_length": max_length,
    "lower": lower,
    "sos_token": sos_token,
    "eos_token": eos_token,
}

train_data = train_data.map(tokenize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(tokenize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(tokenize_example, fn_kwargs=fn_kwargs)

train_data[0]

Map:   0%|          | 0/12800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

{'el': 'Στο σημείο αυτό πρέπει από τώρα να προβλεφθεί μια σημαντική αύξηση των πιστώσεων στο πλαίσιο της αναθεώρησης της συνθήκης ή του κανονισμού.',
 'es': 'Debe considerarse la posibilidad de aumentar considerablemente esta cantidad con ocasión de la revisión del Tratado o del Reglamento.',
 'el_tokens': ['<sos>',
  'στο',
  'σημείο',
  'αυτό',
  'πρέπει',
  'από',
  'τώρα',
  'να',
  'προβλεφθεί',
  'μια',
  'σημαντική',
  'αύξηση',
  'των',
  'πιστώσεων',
  'στο',
  'πλαίσιο',
  'της',
  'αναθεώρησης',
  'της',
  'συνθήκης',
  'ή',
  'του',
  'κανονισμού',
  '.',
  '<eos>'],
 'es_tokens': ['<sos>',
  'debe',
  'considerarse',
  'la',
  'posibilidad',
  'de',
  'aumentar',
  'considerablemente',
  'esta',
  'cantidad',
  'con',
  'ocasión',
  'de',
  'la',
  'revisión',
  'del',
  'tratado',
  'o',
  'del',
  'reglamento',
  '.',
  '<eos>']}

In [14]:
from torchtext.vocab import build_vocab_from_iterator

In [15]:
min_freq = 2
unk_token = "<unk>"
pad_token = "<pad>"

special_tokens = [
    unk_token,
    pad_token,
    sos_token,
    eos_token,
]

el_vocab = build_vocab_from_iterator(
    train_data['el_tokens'],
    min_freq,
    specials=special_tokens,
)

es_vocab = build_vocab_from_iterator(
    train_data["es_tokens"],
    min_freq=min_freq,
    specials=special_tokens,
)

assert el_vocab[unk_token] == es_vocab[unk_token]
assert el_vocab[pad_token] == es_vocab[pad_token]

unk_index = el_vocab[unk_token]
pad_index = es_vocab[pad_token]

print(unk_index)
print(pad_index)

print(el_vocab['είναι'])

0
1
18


In [16]:
el_vocab.set_default_index(unk_index)
es_vocab.set_default_index(unk_index)

In [17]:
def numericalize_example(example, el_vocab, es_vocab):
    el_ids = el_vocab.lookup_indices(example["el_tokens"])
    es_ids = es_vocab.lookup_indices(example["es_tokens"])

    return {"el_ids": el_ids, "es_ids": es_ids}

In [18]:
fn_kwargs = {
    "el_vocab": el_vocab,
    "es_vocab": es_vocab
}

train_data = train_data.map(numericalize_example, fn_kwargs=fn_kwargs)
valid_data = valid_data.map(numericalize_example, fn_kwargs=fn_kwargs)
test_data = test_data.map(numericalize_example, fn_kwargs=fn_kwargs)

train_data[0]

Map:   0%|          | 0/12800 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

{'el': 'Στο σημείο αυτό πρέπει από τώρα να προβλεφθεί μια σημαντική αύξηση των πιστώσεων στο πλαίσιο της αναθεώρησης της συνθήκης ή του κανονισμού.',
 'es': 'Debe considerarse la posibilidad de aumentar considerablemente esta cantidad con ocasión de la revisión del Tratado o del Reglamento.',
 'el_tokens': ['<sos>',
  'στο',
  'σημείο',
  'αυτό',
  'πρέπει',
  'από',
  'τώρα',
  'να',
  'προβλεφθεί',
  'μια',
  'σημαντική',
  'αύξηση',
  'των',
  'πιστώσεων',
  'στο',
  'πλαίσιο',
  'της',
  'αναθεώρησης',
  'της',
  'συνθήκης',
  'ή',
  'του',
  'κανονισμού',
  '.',
  '<eos>'],
 'es_tokens': ['<sos>',
  'debe',
  'considerarse',
  'la',
  'posibilidad',
  'de',
  'aumentar',
  'considerablemente',
  'esta',
  'cantidad',
  'con',
  'ocasión',
  'de',
  'la',
  'revisión',
  'del',
  'tratado',
  'o',
  'del',
  'reglamento',
  '.',
  '<eos>'],
 'el_ids': [2,
  32,
  199,
  28,
  29,
  21,
  144,
  6,
  3689,
  33,
  271,
  355,
  14,
  1157,
  32,
  104,
  8,
  2046,
  8,
  450,
  58,

In [19]:
data_type = "torch"
format_columns = ["el_ids", "es_ids"]

train_data = train_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True
)

valid_data = valid_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

test_data = test_data.with_format(
    type=data_type,
    columns=format_columns,
    output_all_columns=True,
)

train_data


Dataset({
    features: ['el', 'es', 'el_tokens', 'es_tokens', 'el_ids', 'es_ids'],
    num_rows: 12800
})

In [20]:
def get_collate_fn(pad_index):

    def collate_fn(batch):
        batch_el_ids = [example["el_ids"] for example in batch]
        batch_es_ids = [example["es_ids"] for example in batch]
        batch_el_ids = nn.utils.rnn.pad_sequence(batch_el_ids, padding_value=pad_index)
        batch_es_ids = nn.utils.rnn.pad_sequence(batch_es_ids, padding_value=pad_index)
        batch = {
            "el_ids": batch_el_ids,
            "es_ids": batch_es_ids,
        }
        return batch

    return collate_fn


In [21]:
def get_data_loader(dataset, batch_size, pad_index, shuffle=False):

    collate_fn = get_collate_fn(pad_index)

    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        collate_fn=collate_fn,
        shuffle=shuffle,
    )

    return data_loader


In [22]:
# batch_size = 128
batch_size = 16

train_data_loader = get_data_loader(train_data, batch_size, pad_index, shuffle=True)
valid_data_loader = get_data_loader(valid_data, batch_size, pad_index)
test_data_loader = get_data_loader(test_data, batch_size, pad_index)

next(iter(train_data_loader))


{'el_ids': tensor([[   2,    2,    2,  ...,    2,    2,    2],
         [ 167,   88,   12,  ..., 4704,  783,  571],
         [  93,    4,   28,  ...,   19,   26,    6],
         ...,
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1]]),
 'es_ids': tensor([[   2,    2,    2,  ...,    2,    2,    2],
         [ 156,   18,   17,  ..., 1605,   15,   30],
         [ 158,  335,  105,  ...,    4, 1544, 1713],
         ...,
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1],
         [   1,    1,    1,  ...,    1,    1,    1]])}

In [23]:
class Encoder(nn.Module):
    def __init__(self, input_dim, embedding_dim, encoder_hidden_dim, decoder_hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, encoder_hidden_dim, bidirectional = True)
        self.fc = nn.Linear(encoder_hidden_dim * 2, decoder_hidden_dim)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, hidden = self.rnn(embedded)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)))

        return outputs, hidden

In [24]:
class Attention(nn.Module):
    def __init__(self, encoder_hidden_dim, decoder_hidden_dim):
        super().__init__()
        self.attn_fc = nn.Linear(
            (encoder_hidden_dim * 2) + decoder_hidden_dim,
            decoder_hidden_dim
        )
        self.v_fc = nn.Linear(decoder_hidden_dim, 1, bias=False)

    def forward(self, hidden, encoder_outputs):

        batch_size = encoder_outputs.shape[1]
        src_length = encoder_outputs.shape[0]

        hidden = hidden.unsqueeze(1).repeat(1, src_length, 1) 
        encoder_outputs = encoder_outputs.permute(1, 0, 2) 

        energy = torch.tanh(self.attn_fc(torch.cat((hidden, encoder_outputs), dim=2)))

        attention = self.v_fc(energy).squeeze(2) 

        return torch.softmax(attention, dim=1)

In [25]:
class Decoder(nn.Module):
    def __init__(
        self,
        output_dim,
        embedding_dim,
        encoder_hidden_dim,
        decoder_hidden_dim,
        attention,
    ):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, embedding_dim)
        self.rnn = nn.GRU((encoder_hidden_dim * 2) + embedding_dim, decoder_hidden_dim)
        self.fc_out = nn.Linear(
            (encoder_hidden_dim * 2) + decoder_hidden_dim + embedding_dim,
            output_dim
        )

    def forward(self, input, hidden, encoder_outputs):
    
        input = input.unsqueeze(0)
     
        embedded = self.embedding(input)
     
        a = self.attention(hidden, encoder_outputs)

        a = a.unsqueeze(1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
    
        weighted = torch.bmm(a, encoder_outputs) 
        
        weighted = weighted.permute(1, 0, 2)
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
       
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        assert (output == hidden).all()
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim=1))
        
        return prediction, hidden.squeeze(0), a.squeeze(1)



In [26]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio):
        
        batch_size = src.shape[1]
        trg_length = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_length, batch_size, trg_vocab_size).to(self.device)
        
        encoder_outputs, hidden = self.encoder(src)
       
        input = trg[0,:]
        for t in range(1, trg_length):
            
            output, hidden, _ = self.decoder(input, hidden, encoder_outputs)
            
            outputs[t] = output
            
            teacher_force = random.random() < teacher_forcing_ratio
            
            top1 = output.argmax(1)
            
            input = trg[t] if teacher_force else top1
            
        return outputs


In [27]:
input_dim = len(es_vocab)
output_dim = len(el_vocab)
encoder_embedding_dim = 256
decoder_embedding_dim = 256
encoder_hidden_dim = 512
decoder_hidden_dim = 512
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_dir = '/kaggle/working/'

In [28]:
attention = Attention(encoder_hidden_dim, decoder_hidden_dim)

encoder = Encoder(
    input_dim,
    encoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim
)

decoder = Decoder(
    output_dim,
    decoder_embedding_dim,
    encoder_hidden_dim,
    decoder_hidden_dim,
    attention,
)

model = Seq2Seq(encoder, decoder, device).to(device)

model

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10552, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn_fc): Linear(in_features=1536, out_features=512, bias=True)
      (v_fc): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(13009, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=13009, bias=True)
  )
)

In [29]:
def init_weights(m):

    for name, param in m.named_parameters():
        if "weight" in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)

model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10552, 256)
    (rnn): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn_fc): Linear(in_features=1536, out_features=512, bias=True)
      (v_fc): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(13009, 256)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=13009, bias=True)
  )
)

In [30]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=pad_index)

In [31]:
def train_fn(model, data_loader, optimizer, criterion, clip, teacher_forcing_ratio, device):
    model.train()
    epoch_loss = 0
    for i, batch in tqdm(enumerate(data_loader)):
        src = batch["es_ids"].to(device)
        trg = batch["el_ids"].to(device)
       
        optimizer.zero_grad()
        output = model(src, trg, teacher_forcing_ratio)
       
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        
        trg = trg[1:].view(-1)
     
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [32]:
def evaluate_fn(model, data_loader, criterion, device):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            src = batch["es_ids"].to(device)
            trg = batch["el_ids"].to(device)
         
            output = model(src, trg, 0) 
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
           
            trg = trg[1:].view(-1)
            
            loss = criterion(output, trg)
            epoch_loss += loss.item()
    return epoch_loss / len(data_loader)

In [33]:
n_epochs = 10
clip = 1.0 
teacher_forcing_ratio = 0.5

best_valid_loss = float("inf")

for epoch in tqdm(range(n_epochs)):

    train_loss = train_fn(
        model,
        train_data_loader,
        optimizer,
        criterion,
        clip,
        teacher_forcing_ratio,
        device,
    )

    valid_loss = evaluate_fn(
        model,
        valid_data_loader,
        criterion,
        device,
    )

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), os.path.join(model_dir, 'el_es.pt'))

    print(f"\\tTrain Loss: {train_loss:7.3f}")
    print(f"\\tValid Loss: {valid_loss:7.3f}")

  0%|          | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:07,  7.76s/it]
2it [00:21, 11.35s/it]
3it [00:38, 13.77s/it]
4it [00:47, 11.96s/it]
5it [00:52,  9.47s/it]
6it [01:11, 12.72s/it]
7it [01:22, 12.22s/it]
8it [01:50, 17.24s/it]
9it [02:07, 17.13s/it]
10it [02:17, 14.81s/it]
11it [02:21, 11.51s/it]
12it [02:34, 11.92s/it]
13it [02:41, 10.59s/it]
14it [02:53, 11.10s/it]
15it [02:58,  9.19s/it]
16it [03:11, 10.31s/it]
17it [03:14,  8.15s/it]
18it [03:22,  8.04s/it]
19it [03:31,  8.34s/it]
20it [03:39,  8.19s/it]
21it [03:44,  7.19s/it]
22it [03:50,  6.86s/it]
23it [03:54,  6.17s/it]
24it [04:02,  6.72s/it]
25it [04:07,  6.18s/it]
26it [04:15,  6.65s/it]
27it [04:26,  7.81s/it]
28it [04:33,  7.74s/it]
29it [04:43,  8.46s/it]
30it [04:53,  8.88s/it]
31it [05:18, 13.52s/it]
32it [05:23, 11.02s/it]
33it [05:29,  9.50s/it]
34it [05:33,  7.92s/it]
35it [05:44,  8.92s/it]
36it [05:50,  7.97s/it]
37it [05:54,  6.67s/it]
38it [05:59,  6.33s/it]
39it [06:05,  6.36s/it]
40it [06:12,  6

KeyboardInterrupt: 

In [ ]:
model.load_state_dict(torch.load(os.path.join(model_dir, 'el_es.pt'), map_location=device))

test_loss = evaluate_fn(model, test_data_loader, criterion, device)

print(f"| Test Loss: {test_loss:.3f} | Test PPL: {np.exp(test_loss):7.3f} |")

In [ ]:
def translate_sentence(
    sentence,
    model,
    el_nlp,
    es_nlp,
    el_vocab,
    es_vocab,
    lower,
    sos_token,
    eos_token,
    device,
    max_output_length=25,
):
    model.eval()
    with torch.no_grad():
        if isinstance(sentence, str):
            es_tokens = [token.text for token in es_nlp.tokenizer(sentence)]
        else:
            es_tokens = [token for token in sentence]
        if lower:
            es_tokens = [token.lower() for token in es_tokens]
        es_tokens = [sos_token] + es_tokens + [eos_token]
        ids = es_vocab.lookup_indices(es_tokens)
        tensor = torch.LongTensor(ids).unsqueeze(-1).to(device)

        encoder_outputs, hidden = model.encoder(tensor)

        inputs = el_vocab.lookup_indices([sos_token])

        attentions = torch.zeros(max_output_length, 1, len(ids))

        for i in range(max_output_length):
            inputs_tensor = torch.LongTensor([inputs[-1]]).to(device)
            output, hidden, attention = model.decoder(inputs_tensor, hidden, encoder_outputs)
            attentions[i] = attention
            predicted_token = output.argmax(-1).item()
            inputs.append(predicted_token)
            if predicted_token == el_vocab[eos_token]:
                break
        el_tokens = el_vocab.lookup_tokens(inputs)
    return el_tokens, es_tokens, attentions[:len(el_tokens)-1]

In [ ]:
def plot_attention(sentence, translation, attention):
    fig, ax = plt.subplots(figsize=(5,5))
    attention = attention.squeeze(1).numpy()
    cax = ax.matshow(attention, cmap="bone")
    ax.set_xticks(ticks=np.arange(len(sentence)), labels=sentence, rotation=90, size=15)
    translation = translation[1:]
    ax.set_yticks(ticks=np.arange(len(translation)), labels=translation, size=15)
    plt.show()
    plt.close()

In [ ]:
sentence = test_data[0]["es"]
expected_translation = test_data[0]["el"]

sentence, expected_translation

In [ ]:
translation, sentence_tokens, attention = translate_sentence(
    sentence,
    model,
    el_nlp,
    es_nlp,
    el_vocab,
    es_vocab,
    lower,
    sos_token,
    eos_token,
    device,
)

print(translation)
print(sentence_tokens)

In [ ]:
plot_attention(sentence_tokens, translation, attention)

In [ ]:
def plot_loss(train_losses, valid_losses):
    plt.plot(train_losses, label='Training Loss')
    plt.plot(valid_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss vs. Epochs')
    plt.show()

# After training
plot_loss(train_loss, valid_loss)

## Висновок

У цьому проєкті розробила модель машинного перекладу, яка використовує набір даних Europarl для перекладу з грецької мови (el) на іспанську (es). Процес складається з кількох етапів, кожен з яких був критично важливим для успішної реалізації моделі.

### Вибір мовної пари

Я вибрала грецьку (el) та іспанську (es) мови для моделі. Цей вибір базувався на доступності даних та потребі у перекладах між цими мовами, оскільки вони є важливими в рамках Європейського Союзу.

### Завантаження та підготовка даних

На цьому етапі завантажила набір даних Europarl, що містить паралельні тексти на грецькій та іспанській мовах. Потім розділила дані на три набори: тренувальний, валідаційний та тестовий. 

#### Токенізація

Я реалізувала функцію токенізації, яка перетворює текстові рядки в послідовності токенів, що полегшує подальшу обробку тексту моделлю. Токенізація є критично важливою, оскільки вона визначає, як текст буде представлений для моделі.

### Створення моделі

Модель машинного перекладу була побудована на основі архітектури «Encoder-Decoder» із механізмом уваги. 

#### Клас Encoder

Клас `Encoder` обробляє вхідну послідовність та генерує приховані стани. Я використала двонаправлений GRU (Gated Recurrent Unit), що дозволяє моделі враховувати контекст з обох боків послідовності.

#### Клас Attention

Клас `Attention` відповідає за обчислення контекстного вектора, який використовується декодером для генерації перекладу. Він забезпечує, щоб модель могла зосереджуватись на важливих частинах вхідного тексту під час перекладу.

#### Клас Decoder

Клас `Decoder` використовує контекстний вектор та генерує вихідні токени (перекладені слова). Він спочатку отримує токен із вхідної послідовності, а потім генерує наступний токен, спираючись на попередній контекст.

#### Клас Seq2Seq

Модель `Seq2Seq` об'єднує енкодер та декодер у єдину модель, здатну обробляти вхідні та вихідні послідовності. Це дозволяє моделі працювати в контексті повного перекладу.

### Функція втрат та оптимізатор

Я використала `CrossEntropyLoss` як функцію втрат, що є стандартним підходом для задач класифікації. Оптимізатор Adam був обраний через його ефективність у навчанні глибоких нейронних мереж.

### Навчання моделі

Модель була навчена протягом 10 епох. Я реалізувала цикл навчання, який включав обчислення втрат на тренувальному наборі даних та валідаційному наборі. 

#### Збереження найкращої моделі

Я зберегла модель, яка показала найнижчі втрати на валідаційних даних. Це допомогло уникнути перенавчання та зберегти модель з найкращими показниками.

### Візуалізація

Після завершення навчання візуалізувала втрати тренування та валідації, щоб проаналізувати продуктивність моделі. Цей графік дозволяє спостерігати, чи зменшуються втрати з часом, що є хорошим показником успішного навчання.

### Візуалізація механізму уваги

Для візуалізації механізму уваги використала виходи механізму уваги з декодера. Це дозволиляє побачити, на які слова вхідної послідовності модель звертає увагу при генеруванні кожного слова у вихідній послідовності.

### Приклади перекладу

Протестувала модель, запитуючи переклади для речень. Це дало можливість перевірити, як добре модель може виконувати поставлену задачу, і виявити можливі помилки або недоліки.

### Загальний результат

Модель успішно навченa для виконання перекладу з грецької на іспанську мову. Хоча вона потребує подальшого вдосконалення, результати вже демонструють обнадійливі показники. Додаткові налаштування гіперпараметрів та підвищення обсягу тренувальних даних можуть призвести до покращення якості перекладу.
